# Case
> Требуется улучшить этап отбора кандидатов в поисковой веб-системе. На текущий момент в качестве кандгена (кандидатогенерации) используется BM25 и обратный индекс. BM25 уже тюнили, дальше качество нарастить не выходит. В качестве бизнес-метрики можем взять производные поведенческого отклика, например, CTR@K или timespent на выдаче и документах.

Очевидным направлением развития является построение нейросетевого кандгена. Обычно в описанных случаях действуют следующим образом:
0. Выбирают ML-метрику, которую хотелось бы оптимизировать. Для кандгена катастрофически важно выдать как можно больше релеватных документов в пределах фиксированной длины выдачи, поэтому подходящая метрика -- Recall@K. Мы будем использовать ее модификацию, но об этом позже.
1. Сэмплируют запросы из потока / формируют специфичные корзины запросов в зависимости от дополнительных бизнес-требований. Давайте считать, что они отсутствуют. Тут обязателен контроль их качества, можно исходить из символьных эвристик или применять LLM для классификации, как вы это делали в предыдущей домашке.
2. Обкачивают поисковый движок, формируя глубокие выдачи. Эпитет "глубокие" относится к глубине погружения пользователя в выдачу, то есть предельные позиции взаимодействия с документами. Так вот для обучения требуется брать документов в избытке, в том числе те, с которыми пользователь никогда бы не повзаимодействовал. В целом, длина выдачи 1000 -- отличный выбор. Предварительно есть смысл сгладить все условия отбора по BM25.
3. Разметка пар запрос-документ на задачу релевантности. LLM -- вновь отличный выбор. Разметка порядковая, но может быть как бинарной, так и n-арной. Важно сформировать определение "релевантного" документа, то есть определить порог, по которому мы будем считать документ подходящим под запрос.
4. Релевантные пары запрос-документ берем в качестве позитивов, выбираем базовый эмбеддер и учим его контрастивно как bi-энкодер на эту выборку, негативы можем формировать в режиме in-batch.
5. Если все сделано верно (данных достаточно, гиперпараметры подобраны, код не багованный), естественным следствием будет рост качества.

Датасет, на котором мы будем строить кандген -- MS Marco Dev. 

Библиотека для работы с датасетами - `ir_datasets` ([API](https://ir-datasets.com/python.html)). "IR" от Information Retrieval - библиотека содержит инструменты работы с датасетами поиска.  
В коде будет использоваться `polars` ([API](https://docs.pola.rs/api/python/stable/reference/index.html)), аналог `pandas`, только на порядки быстрее.

[Описание датасета](https://ir-datasets.com/msmarco-passage.html#msmarco-passage/dev/judged).

### Dependencies

In [ ]:
!pip install ir-datasets -q
!pip install faiss-cpu -q

In [ ]:
import re
import faiss
import random
import numpy as np
import ir_datasets
import polars as pl
from tqdm import tqdm
from functools import partial
from dataclasses import dataclass
from collections import defaultdict

import torch
from torch import nn
from torch.optim import AdamW
from torch.amp import GradScaler
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import OneCycleLR, CosineAnnealingLR

from transformers import AutoModel, AutoTokenizer, AutoConfig
from sklearn.model_selection import train_test_split

### Data

In [ ]:
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

In [ ]:
@dataclass
class Columns:
    query_id: str = "query_id"
    doc_id: str = "doc_id"
    index_id: str = "index_id"
    text: str = "text"
    qrels_relevance: str = "relevance"


@dataclass
class DatasetConfig:
    sampled_index_size: int = 150_000
    relevance_threshold: int = 1
    test_size: float = 0.2

In [ ]:
dataset = ir_datasets.load("msmarco-passage/dev/judged")

columns = Columns()
dataset_config = DatasetConfig()

In [ ]:
queries = pl.DataFrame(dataset.queries_iter()).select(
    pl.col(columns.query_id).cast(pl.Int32),
    pl.col(columns.text)
)

qrels = pl.DataFrame(dataset.qrels_iter()).drop("iteration").select(
    pl.col(columns.query_id).cast(pl.Int32),
    pl.col(columns.doc_id).cast(pl.Int32),
    pl.col(columns.qrels_relevance).cast(pl.Int32)
)

documents = pl.DataFrame(dataset.docs_iter()).select(
    pl.col(columns.doc_id).cast(pl.Int32),
    pl.col(columns.text)
)

In [ ]:
target_document_ids = qrels[columns.doc_id].unique().to_list()
sampled_document_ids = np.random.default_rng().integers(dataset.docs_count(), size=dataset_config.sampled_index_size).tolist()

sampled_documents = documents.filter(pl.col(columns.doc_id).is_in(sampled_document_ids + target_document_ids)).with_row_index(columns.index_id)
len(target_document_ids), len(sampled_document_ids)

In [ ]:
query_ids = qrels[columns.query_id].to_numpy()
doc_ids = qrels[columns.doc_id].to_numpy()

# Разбиваем индексы
train_idx, test_idx = train_test_split(
    np.arange(len(qrels)),
    test_size=dataset_config.test_size,
    random_state=42
)

train_qrels = qrels[train_idx]
test_qrels = qrels[test_idx]

In [ ]:
train_queries = queries.filter(pl.col(columns.query_id).is_in(train_qrels[columns.query_id].to_list()))
test_queries = queries.filter(pl.col(columns.query_id).is_in(test_qrels[columns.query_id].to_list()))

train_documents = sampled_documents.filter(pl.col(columns.doc_id).is_in(train_qrels[columns.doc_id].to_list()))
test_documents = sampled_documents.filter(pl.col(columns.doc_id).is_in(test_qrels[columns.doc_id].to_list()))

### Training config

In [ ]:
@dataclass
class TrainTestConfig:
    device: str = torch.device(["cpu", "cuda"][torch.cuda.is_available()])
    batch_size: int = 32
    max_query_len: int = 32
    max_doc_len: int = 128
    sampled_index_size: int = 150_000
    recalls_k: list = (1, 5, 10, 50)
    acc_steps: int = 4
    epochs: int = 3
    relevance_threshold: int = 1

@dataclass
class ModelConfig:
    model_name: str = "microsoft/deberta-v3-small"
    agg: str = "mean"
    freeze_base: bool = False
    out_dim: int = 256

@dataclass
class LossConfig:
    thrsh: float = 0.1
    temperature: float = 0.05

In [ ]:
columns = Columns()
dataset_config = DatasetConfig()
train_test_config = TrainTestConfig()

loss_config = LossConfig()
model_config = ModelConfig()

### Dataset

`DenseRetrievalDataset` - датасет, который внутри формирует множество релевантных пар и выдает на каждый индекс произвольную пару оттуда вместе с `query_id` и `doc_id`.

`train_collate_fn` - функция, которая внутри токенизирует батчем текст запроса и документа и отдает кортеж из тензоров, в которые включаются id запросов и документов, токены запросов и документов.

In [ ]:
class DenseRetrievalDataset(Dataset):
    def __init__(self, queries, documents, qrels, columns, config):
        self.columns = columns
        self.config = config

        self.queries = queries
        self.documents = documents
        self.qrels = qrels.filter(
            pl.col(columns.qrels_relevance) >= config.relevance_threshold
        )
        self.query_texts = dict(zip(
            queries[columns.query_id].to_list(),
            queries[columns.text].to_list()
        ))

        self.doc_texts = dict(zip(
            documents[columns.doc_id].to_list(),
            documents[columns.text].to_list()
        ))

        self.relevant = {}
        for row in self.qrels.iter_rows(named=True):
            query_id = row[columns.query_id]
            doc_id = row[columns.doc_id]
            if query_id in self.query_texts and doc_id in self.doc_texts:
                self.relevant.setdefault(query_id, []).append(doc_id)

        self.query_ids = list(self.relevant.keys())

    def __len__(self):
        return len(self.query_ids)

    def __getitem__(self, idx):
        query_id = self.query_ids[idx]

        doc_id = random.choice(self.relevant[query_id])
        return {
            "query_id": query_id,
            "doc_id": doc_id,
            "query_text": self.query_texts[query_id],
            "doc_text": self.doc_texts[doc_id]
        }

In [ ]:
def train_collate_fn(data, tokenizer, config):
    query_texts = [item["query_text"] for item in data]
    doc_texts = [item["doc_text"] for item in data]
    query_ids = [item["query_id"] for item in data]
    doc_ids = [item["doc_id"] for item in data]

    query_tokens = tokenizer(
        query_texts,
        padding="max_length",
        truncation=True,
        max_length=config.max_query_len,
        return_tensors="pt"
    )

    doc_tokens = tokenizer(
        doc_texts,
        padding="max_length",
        truncation=True,
        max_length=config.max_doc_len,
        return_tensors="pt"
    )

    return (
        torch.tensor(query_ids),
        torch.tensor(doc_ids),
        query_tokens,
        doc_tokens
    )

### Loss Function (InfoNCE + In-batch negatives mining)

`ContrastiveLoss`, который реализует расчет следующей функции ошибки:
$$\mathcal{L}=\mathbb{E}_T\text{CrossEntropy}\left(q_iD^T-B_i, M_i\right)$$
$$T=\{Q, D\},\quad Q=\{q_i\big|q_i\in\mathbb{R}^n,\|q_i\|_2=1\}_{i=1}^N,\quad D=\{d_i\big|d_i\in\mathbb{R}^n,\|d_i\|_2=1\}_{i=1}^N$$
$$(q_i, d_i) \,-\,\text{позитивная пара}$$
$$M_i\in[0,1]^N,\quad \forall{j}\in\overline{1,N}:\;M[j]=\frac{[q_i = q_j]}{\sum\limits_k{[q_i=q_k]}}$$
$$B_i\in[0,1]^N,\quad \forall{j}\in\overline{1,N}:\;M[j]=b*[q_i = q_j]$$
$$b\,-\,\text{вещественный гиперпараметр}$$

Смысл $b$ - [статья LaBSE](https://arxiv.org/pdf/2007.01852), _Additive Margin Softmax_. (штрафуем позитивы, вынуждая модель еще больше поднимать косинус, чтобы компенсировать штраф)

In [ ]:
class ContrastiveLoss(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.thrsh = cfg.thrsh # то самое b
        self.temperature = cfg.temperature  # τ

    def forward(self, queries, documents, labels):
        q = F.normalize(queries,  dim=1, p=2)
        d = F.normalize(documents, dim=1, p=2)

        logits = (q @ d.T) / self.temperature

        logits = logits - self.thrsh * (labels > 0).float()

        log_probs = F.log_softmax(logits, dim=1)
        loss = -(labels * log_probs).sum(dim=1).mean()
        return loss

In [ ]:
criterion = ContrastiveLoss(loss_config)

### Model

In [ ]:
class Embedder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.agg = config.agg.lower()
        cfg = AutoConfig.from_pretrained(config.model_name, output_hidden_states=False)
        self.backbone = AutoModel.from_pretrained(config.model_name, config=cfg)

        if config.freeze_base:
            for p in self.backbone.parameters():
                p.requires_grad = False

        self.head = nn.Linear(self.backbone.config.hidden_size, config.out_dim, bias=False)


    def forward(self, input):
        x = self.backbone(**input).last_hidden_state
        if self.agg == 'cls':
            x = x[:, 0]
        elif self.agg == 'mean':
            mask = input["attention_mask"].unsqueeze(-1).float()
            x = (x * mask).sum(1) / mask.sum(1).clamp(min=1e-6)
        x = self.head(x)
        return F.normalize(x, p=2, dim=1)

### Test metric and inference

Пайплайн для оценки работы retriever с использованием модифицированной метрики Recall@K на тестовой выборке.

##### 1. Функция `inference`

Выполняет прогон эмбеддера по списку текстов.

* Для запросов (`is_query=True`) и документов (`is_query=False`) используется разная максимальная длина токенизации.
* Тексты разбиваются на батчи и токенизируются.
* Эмбеддер переводится в режим `eval` и на заданное устройство (`config.device`).
* Вычисленные векторы собираются и возвращаются в виде одного тензора.

##### 2. Функция `calc_recall`

Вычисляет усреднённый модифицированный `Recall@K` для набора запросов.

* Из FAISS-индекса извлекаются top-K ближайших документов для каждого запроса.
* ID документов из индекса сопоставляются с исходными документами.
* Для каждого запроса считается доля релевантных документов в выдаче на уровне каждого `K` из списка `config.recalls_k`.
* Итоговое значение — среднее по всем валидным запросам.

Формула:

$$
Recall@K = \frac{\text{\# релевантных документов в top-K}}{\min(\text{\# всех релевантных}, \text{\# документов в выдаче})}
$$

##### 3. Вспомогательная функция `_ensure_index_id`

Гарантирует наличие уникального поля с индексом документа (`index_id`) в DataFrame.

##### 4. Функция `test_retriever`

Запускает полный процесс оценки:

1. При необходимости уменьшает набор документов для ускорения теста.
2. Генерирует эмбеддинги документов и нормализует их (`faiss.normalize_L2`).
3. Строит FAISS-индекс по эмбеддингам документов.
4. Генерирует эмбеддинги запросов и нормализует их.
5. Передаёт индекс, векторы запросов и данные релевантности (`qrels`) в `calc_recall`.
6. Возвращает словарь с `Recall@K` для всех заданных значений `K`.


In [ ]:
@torch.inference_mode()
def inference(embedder, texts, is_query, config, tokenizer):
    max_len = config.max_query_len if is_query else config.max_doc_len

    dl = DataLoader(texts, batch_size=config.batch_size, shuffle=False)
    all_vecs = []

    embedder.eval()
    embedder.to(config.device)

    for batch in tqdm(dl, desc="Inference", total=len(dl)):
        tokenized_batch = tokenizer(
            batch,
            padding="max_length",
            truncation=True,
            max_length=max_len,
            return_tensors="pt"
        )
        tokenized_batch = {k: v.to(config.device) for k, v in tokenized_batch.items()}
        vec = embedder(tokenized_batch)
        all_vecs.append(vec.cpu())

    return torch.cat(all_vecs, dim=0)

In [ ]:
def calc_recall(index, query_embeddings, query_ids, qrels, documents, columns, config):
    Ks = sorted(config.recalls_k)
    max_k = Ks[-1]

    query_embs_np = query_embeddings.astype("float32", copy=False)

    _, idx_mat = index.search(query_embs_np, max_k)

    index_ids = documents[columns.index_id].to_numpy()
    doc_ids = documents[columns.doc_id].to_numpy()
    id_map = dict(zip(index_ids, doc_ids))

    doc_id_mat = np.vectorize(id_map.get)(idx_mat, -1)

    rel_dict = defaultdict(set)
    for row in qrels.iter_rows(named=True):
        if row[columns.qrels_relevance] >= config.relevance_threshold:
            rel_dict[row[columns.query_id]].add(row[columns.doc_id])

    recall_sum = {k: 0.0 for k in Ks}
    valid_queries = 0

    for row_idx, query_id in enumerate(query_ids):
        relevant = rel_dict.get(query_id)
        if not relevant:
            continue

        retrieved_ids = doc_id_mat[row_idx]
        valid_queries += 1

        for k in Ks:
            topk = set(retrieved_ids[:k])
            topk.discard(-1)

            intersection = topk & relevant
            denom = min(len(relevant), len(topk))
            recall_sum[k] += len(intersection) / denom if denom else 0.0

    recall_mean = {k: (recall_sum[k] / valid_queries if valid_queries else 0.0) for k in Ks}
    return recall_mean

In [ ]:
def _ensure_index_id(df: pl.DataFrame, name):
    if name in df.columns:
        if df[name].n_unique() != df.height:
            df = df.with_columns(pl.arange(0, df.height).alias(name))
        return df
    else:
        return df.with_row_index(name, offset=0)


def test_retriever(embedder, test_queries, test_qrels, documents, columns,
                   config, tokenizer, use_small_eval_set=False, subset_size=500):
    if use_small_eval_set:
        docs_eval = documents[:subset_size]
    else:
        docs_eval = documents
    docs_df = _ensure_index_id(docs_eval, columns.index_id)

    doc_vecs = inference(
        embedder,
        docs_df[columns.text].to_list(),
        is_query=False,
        config=config,
        tokenizer=tokenizer
    ).cpu().numpy().astype("float32")

    faiss.normalize_L2(doc_vecs)

    dim = doc_vecs.shape[1]
    index = faiss.IndexFlatIP(dim)

    index.add(doc_vecs)

    q_vecs = inference(
        embedder,
        test_queries[columns.text].to_list(),
        is_query=True,
        config=config,
        tokenizer=tokenizer
    ).cpu().numpy().astype("float32")

    faiss.normalize_L2(q_vecs)

    q_ids = test_queries[columns.query_id].to_numpy()

    recall = calc_recall(
        index=index,
        query_embeddings=q_vecs,
        query_ids=q_ids,
        qrels=test_qrels,
        documents=docs_df,
        columns=columns,
        config=config
    )
    return recall

### Training function + Gradient accumulation

Готовим лоадеры, обучаем модель в контрастимном режиме и считаем метрику раз в эпоху

In [ ]:
def train_retriever(
    embedder, train_queries, train_documents,
    train_qrels, test_queries, test_qrels,
    documents, columns, dataset_config,
    train_test_config, loss_config, tokenizer
):
    train_ds = DenseRetrievalDataset(
        train_queries, train_documents, train_qrels,
        columns, dataset_config
    )

    train_dl = DataLoader(
        train_ds,
        batch_size=train_test_config.batch_size,
        shuffle=True,
        num_workers=4,
        pin_memory=True,
        collate_fn=lambda b: train_collate_fn(b, tokenizer, train_test_config),
        drop_last=True
    )

    optim  = AdamW(
        filter(lambda p: p.requires_grad, embedder.parameters()),
        lr=2e-5,
        weight_decay=0.01
    )
    scaler = GradScaler()
    loss_fn = ContrastiveLoss(loss_config)

    embedder.to(train_test_config.device).train()

    for ep in range(1, train_test_config.epochs + 1):
        pbar = tqdm(train_dl, desc=f"E{ep}")
        running = 0.0
        optim.zero_grad(set_to_none=True)

        for step, batch in enumerate(pbar, start=1):
            q_ids, d_ids, q_tok, d_tok = batch
            q_tok = {k: v.to(train_test_config.device) for k, v in q_tok.items()}
            d_tok = {k: v.to(train_test_config.device) for k, v in d_tok.items()}

            with torch.autocast("cuda" if torch.cuda.is_available() else "cpu"):
                q_vec = embedder(q_tok)
                d_vec = embedder(d_tok)

                labels = torch.eye(q_vec.size(0), device=train_test_config.device)

                loss_qd = loss_fn(q_vec, d_vec, labels)
                loss_dq = loss_fn(d_vec, q_vec, labels)

                loss = 0.5 * (loss_qd + loss_dq) / train_test_config.acc_steps

            scaler.scale(loss).backward()
            running += loss.item() * train_test_config.acc_steps

            if step % train_test_config.acc_steps == 0:
                scaler.unscale_(optim)
                torch.nn.utils.clip_grad_norm_(embedder.parameters(), 1.0)
                scaler.step(optim)
                scaler.update()
                optim.zero_grad(set_to_none=True)

            pbar.set_postfix(loss=f"{running/step:.4f}")

        embedder.eval()
        with torch.no_grad():
            scores = test_retriever(
                embedder, test_queries, test_qrels,
                documents, columns, train_test_config, tokenizer
            )
        print(f"Epoch {ep}  Recall: {scores}")
        embedder.train()

    return scores

### Final

Обучаем эмбеддер и сравниваем метрики

In [ ]:
embedder = Embedder(model_config)
tokenizer = AutoTokenizer.from_pretrained(model_config.model_name)
embedder.to(train_test_config.device)

In [ ]:
embedder.eval()
baseline_recall = test_retriever(
    embedder=embedder,
    test_queries=test_queries,
    test_qrels=test_qrels,
    documents=sampled_documents,
    columns=columns,
    config=train_test_config,
    tokenizer=tokenizer,
    use_small_eval_set=False,
    subset_size=500
)
print("Baseline Recall:", baseline_recall)

In [ ]:
final_recall = train_retriever(
    embedder=embedder,
    train_queries=train_queries,
    train_documents=train_documents,
    train_qrels=train_qrels,
    test_queries=test_queries,
    test_qrels=test_qrels,
    documents=sampled_documents,
    columns=columns,
    dataset_config=dataset_config,
    train_test_config=train_test_config,
    loss_config=loss_config,
    tokenizer=tokenizer
)

print("Recall после обучения:", final_recall)

In [ ]:
embedder.eval()

after_recall = test_retriever(
    embedder=embedder,
    test_queries=test_queries,
    test_qrels=test_qrels,
    documents=sampled_documents,
    columns=columns,
    config=train_test_config,
    tokenizer=tokenizer,
    use_small_eval_set=False,
    subset_size=500
)

print("Recall до обучения:", baseline_recall)
print("Recall после обучения:", after_recall)

for k in train_test_config.recalls_k:
    delta = after_recall[k] - baseline_recall[k]
    print(f"Delta Recall@{k:<2}: {delta:+.3f}")